# 🧪 Lab 6: Tools & Tool-Calling Agents with LangChain

**Builds on Lab 5**: You learned LLMs, `PromptTemplate`, and `LLMChain`.  
**Now**: Wrap external functions as **Tools** and let an **Agent** decide when to use them.

## 🎯 Learning Outcomes
- Define LangChain **Tools** with the `@tool` decorator.
- Understand two agent styles: **Tool-Calling** (OpenAI functions) and **ReAct**.
- Run end-to-end demos that combine tools + LLM reasoning.
- Connect this to the Agentic AI lifecycle from Labs 1–4.


## 🛠 Setup

Install dependencies and set your API key:

```bash
pip install langchain openai
# mac/linux:
export OPENAI_API_KEY="your_key"
# windows powershell:
setx OPENAI_API_KEY "your_key"
```


In [ ]:
from typing import Dict
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import initialize_agent, AgentType


## 1) Define Tools

We keep tools **offline-friendly** for predictable training:
- `get_weather(city)` → tiny in-memory weather DB
- `calculator(expression)` → safe arithmetic
- `mini_wiki(topic)` → tiny offline encyclopedia


In [ ]:
@tool("get_weather", return_direct=False)
def get_weather(city: str) -> str:
    """Return simple weather for a given city from a tiny offline database."""
    weather_db: Dict[str, str] = {
        "paris": "sunny, 24°C",
        "chicago": "cloudy, 18°C",
        "mumbai": "rainy, 30°C",
        "london": "overcast, 17°C",
        "tokyo": "clear, 26°C",
    }
    c = city.strip().lower()
    if c in weather_db:
        return f"The weather in {city} is {weather_db[c]}."
    return f"No weather found for '{city}'. Assume mild (22°C) and clear for demo."


In [ ]:
@tool("calculator", return_direct=False)
def calculator(expression: str) -> str:
    """Safely evaluate a simple math expression (digits and + - * / ( ) . %)."""
    import math, re
    if not re.fullmatch(r"[0-9+\-*/(). %\s]+", expression):
        return "Calculator error: invalid characters."
    try:
        result = eval(expression, {"__builtins__": {}}, {"math": math})
        return str(result)
    except Exception as e:
        return f"Calculator error: {e}"


In [ ]:
@tool("mini_wiki", return_direct=False)
def mini_wiki(topic: str) -> str:
    """Return a short summary from a tiny offline encyclopedia."""
    kb: Dict[str, str] = {
        "alan turing": "Alan Turing (1912–1954) was a mathematician and pioneer of computer science who formalized computation and contributed to codebreaking in WWII.",
        "agentic ai": "Agentic AI refers to systems that can plan, choose tools/actions, and adapt using feedback, rather than only producing text responses.",
        "langchain": "LangChain is a framework that provides building blocks for LLM apps: prompts, chains, tools, memory, and agents.",
    }
    return kb.get(topic.strip().lower(), "No entry found in mini_wiki. Try 'Alan Turing', 'Agentic AI', or 'LangChain'.")


## 2) Create an LLM


In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = [get_weather, calculator, mini_wiki]


## 3) Agent Style A — Tool-Calling (OpenAI Functions)

This agent uses the model's native **function-calling** to choose and call tools.


In [ ]:
tool_calling_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)


In [ ]:
queries = [
    "What's the weather in Paris right now? Suggest one outdoor activity.",
    "Calculate: 23*17 + 3.5, then divide by 2.",
    "Use mini_wiki to summarize 'Alan Turing' in one sentence, then list two contributions."
]
for q in queries:
    print("\nUser:", q)
    result = tool_calling_agent.invoke({"input": q})
    print("Assistant:", result["output"])


## 4) Agent Style B — ReAct (Reasoning & Acting)

ReAct agents follow a **Thought → Action → Observation** loop and work well with tool descriptions.


In [ ]:
react_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
q = "Plan an evening in Chicago based on the weather. Include one indoor and one outdoor option."
print("User:", q)
result = react_agent.invoke({"input": q})
print("Assistant:", result["output"])


## 🔄 How This Maps to Labs 1–4

- **Tools** → Replaces our `tools.py` functions and manual wiring.
- **Agent** → Replaces our hand-coded decision loop.
- **LLM wrapper** → Replaces our raw API client and retry logic.

Same Agentic AI lifecycle; less boilerplate and easier to extend.
